In [5]:
import pandas as pd
from tqdm import tqdm
import os
import json
import cv2
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def get_x_from_y(m, c, y=700):
    return (y - c) / m

def get_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    desired_width = 1280
    desired_height = 720
    if (cap.isOpened() == False): 
        print("Error opening video stream or file")

    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, (desired_width, desired_height))
            frames.append(frame)
        else:
            break
    cap.release()
    return frames , fps , desired_width , desired_height

def get_lane_line_after_linear_regression(frame_data):
    frame_lane_keyps_arr = np.array(frame_data['lane_keyps'][0])
    inds = range(0, frame_lane_keyps_arr.shape[1] + 1, 40)
    inds = list(zip(*[inds[:-1], inds[1:]]))
    # Store the linear regression models for each lane
    lane_models = dict()  



    for lane_ind in range(len(inds)):
        ################################# for Storing the x and y for the lane #######################################

        x_coords = []
        y_coords = []

        for keyp_i in range(*inds[lane_ind]):
            point = tuple(map(lambda flt: int(round(flt, 0)), frame_lane_keyps_arr[:, keyp_i]))
            # if point[0] != 0:
            x_coords.append(point[0])
            y_coords.append(point[1])

        ################################# Running Linear Regression to get the best fit line from the points #######################################

        if lane_ind == 1 or lane_ind == 2:
            # Linear regression to find the best fit line for the points
            x_coords = np.array(x_coords).reshape(-1, 1)
            y_coords = np.array(y_coords)

            model = LinearRegression()
            model.fit(x_coords, y_coords)
            m , c = model.coef_[0], model.intercept_
            lane_models[lane_ind] = (m,c)  # Store the model
    return lane_models


def plot_points(frame, lane_models, center_of_frame):
    m1, c1 = lane_models[1]
    m2, c2 = lane_models[2]
    x_truck_left = get_x_from_y(m1, c1)
    x_truck_right = get_x_from_y(m2, c2)

    # Draw a white circle at the center of the frame
    frame = cv2.circle(frame, (center_of_frame, 700), 5, (255, 255, 255), thickness=-1)

    # Draw a white circle at the x_truck_right position
    frame = cv2.circle(frame, (int(x_truck_right), 700), 5, (255, 255, 255), thickness=-1)

    # Draw a blue line from center_of_frame to x_truck_right
    frame = cv2.line(frame, (center_of_frame, 700), (int(x_truck_right), 700), (255, 0, 0), thickness=2)

    return frame


def main(video_paths : list ,source_path_json : str,result_path : str):
    LANE_COLOURS = {0: (0, 0, 0), 1: (0, 0, 255), 2: (255, 0, 0), 3: (255, 255, 255)}
    window_size = 6
    downsample_fps = 6
    frame_step_size = int(30 / downsample_fps)
    for _, video_path in enumerate(tqdm(video_paths)):
        
        video_offline_id = video_path.split('/')[-1].replace('_viz.mp4','').replace('.mp4','')[13:]
        video_json_path = source_path_json + video_path.split('/')[-1].replace('_viz.mp4','.json').replace('.mp4','.json')

        # Path to save the video
        video_with_mid_point_logic = result_path + '/' + "{}.mp4".format(video_offline_id)

        # Reading Json File
        with open(video_json_path, 'r') as fh:
            lane_dict = json.load(fh)
        frames , fps , desired_width, desired_height = get_frames(video_path=video_path)

        assert len(frames) == len(lane_dict['frames']), "mismatch in number of video frames and number of lane keyps frames"

        out_mid_point = cv2.VideoWriter(video_with_mid_point_logic, cv2.VideoWriter_fourcc(*'mp4v'), downsample_fps, (desired_width, desired_height))


        number_of_frames = int(len(frames) / frame_step_size)
        for frame_number , (frame, frame_ind) in enumerate(zip(frames, lane_dict['frames'])):
            # this control the fps rate 
            if frame_number % frame_step_size != 0:
                continue

            frame_data = lane_dict['frames'][frame_ind]
            lane_models = get_lane_line_after_linear_regression(frame_data)
            # Compute intersection of the lines of lane 1 and lane 2 to find vanishing point
            try:
                
                mid_point_frame  = plot_points(frame.copy(),lane_models , 640)
            except Exception as e:
                # print(e)
                pass
            out_mid_point.write(mid_point_frame)

        out_mid_point.release()

In [7]:
import pandas as pd 
df = pd.read_csv('./shortlisted_vids - Sheet1.csv')
offline_id = df['offline_id'].tolist()
video_paths  = list()
for o_i in offline_id:
    video_paths.append(
        f'/home/muhammadasadsoomro/Desktop/Motive/Lane_Assist/Datasets/drowsiness_distacted/videos/front_facing-{o_i}.mp4'
    )
video_paths
results_data = main(video_paths=video_paths , 
     source_path_json='/home/muhammadasadsoomro/Desktop/Motive/Lane_Assist/Datasets/drowsiness_distacted/final_dataset/v_jsons/' ,
     result_path='/home/muhammadasadsoomro/Desktop/Work/LDW/ball_visualizations_for_devin/videos/')

100%|██████████| 15/15 [00:48<00:00,  3.26s/it]


In [2]:
df = pd.read_csv('./shortlisted_vids - Sheet1.csv')
df

,offline_id,video_link,category,comments,Unnamed: 4
0,1b34ca14-5fba-4667-a52a-58284050b323,https://drive.google.com/file/d/1Om-nDvtkP5YrO...,low,NaN,Migh be the lane Line issue
1,8a9dc84b-25df-4a05-a4d7-21759818ff6a,https://drive.google.com/file/d/1TMcEl-vCvuAiP...,med,"good med, weak high",NaN
2,3e35603d-bca2-403e-9c3b-197e4ee1e145,https://drive.google.com/file/d/1AXCpeDwFA9yXG...,med,NaN,This might be because of the Lane Lines
3,68a6562f-31eb-4625-b895-b02b4afcfb3e,https://drive.google.com/file/d/1e0773TKgbtL3E...,med,maybe high,NaN
4,00fbe656-7517-4841-bba8-d31196d92d2d,https://drive.google.com/file/d/1CNWO0eVK_9-WY...,med,"definitely FP, letting vehicles pass",NaN
5,eba6f967-0afa-433e-8835-09622901f066,https://drive.google.com/file/d/1DUymOFijgKvLb...,low,NaN,NaN
6,bf884f11-2d12-4c70-8f73-c17aa4d46432,https://drive.google.com/file/d/1BQ8ehCDBBRcno...,low,very low,NaN
7,d9c37407-6235-480d-a790-563635d75b70,https://drive.google.com/file/d/1M0dxhqKMtyK9w...,low,slight wiggle,NaN
8,37276a30-44c9-454f-bc55-ce64f304a23f,https://drive.google.com/file/d/1xFCVzFphl-8Ra...,low,NaN,NaN
9,16519612-0e86-4973-bd30-52669c38b52a,https://drive.google.com/file/d/127I3SmEutO3YF...,low,"maybe medium, special case of turning",I would have my vote on the low as the slight ...
